# 7.2 Different ways to build Keras models

## 7.2.1 The Sequential model

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])

# Or alternatively,
model = keras.Sequential()
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))

**A model has no weights until it is called on some data. We can do this with its `build()` method**

In [4]:
# This causes an arrow as the model has not been called yet
model.weights

ValueError: Weights for model sequential_2 have not yet been created. Weights are created when the Model is first called on inputs or `build()` is called with an `input_shape`.

In [5]:
model.build(input_shape=(None, 3))

# No error. Input shape is resolve due to calling build()
model.weights

[<tf.Variable 'dense_4/kernel:0' shape=(3, 64) dtype=float32, numpy=
 array([[-0.046359  , -0.01522747, -0.19727807,  0.03270653, -0.2849747 ,
          0.21612722, -0.02864486,  0.28301865,  0.28882617, -0.22891366,
          0.13251844, -0.19033232,  0.19589594, -0.03387463, -0.07952063,
         -0.13838947,  0.10327926,  0.05222139,  0.17876548, -0.26505286,
          0.05783302, -0.24561937, -0.11805038, -0.00544018, -0.20787644,
          0.05869839,  0.06419307,  0.00565743, -0.25582305,  0.03097913,
          0.10474509, -0.12859347,  0.0257214 ,  0.01292437,  0.16990784,
         -0.11713828,  0.00613809, -0.08294702, -0.04911643,  0.1504947 ,
         -0.2945595 ,  0.2633143 ,  0.15403026, -0.13624611, -0.15009351,
          0.16270801,  0.00156906,  0.18423569, -0.2675587 , -0.04487762,
          0.11733335,  0.25730175,  0.22899473,  0.23653823,  0.05483907,
         -0.16598044,  0.00605732,  0.07338783,  0.0707626 ,  0.06388593,
          0.14155859,  0.00955364,  0.17534

In [7]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                256       
                                                                 
 dense_5 (Dense)             (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


**Models can be named**

In [8]:
model = keras.Sequential(name="my_example_model")
model.add(layers.Dense(64, activation="relu", name="my_first_layer"))
model.add(layers.Dense(10, activation="softmax", name="my_last_layer"))

model.build((None, 3))
model.summary()

Model: "my_example_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_first_layer (Dense)      (None, 64)                256       
                                                                 
 my_last_layer (Dense)       (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


**Have your Sequential models be built on the fly be specifying an input layer**

In [10]:
model = keras.Sequential()
model.add(keras.Input(shape=(3,)))
model.add(keras.layers.Dense(64, activation="relu"))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 64)                256       
                                                                 
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


## 7.2.2 The Functional API

**The `Sequential` model is limited by the fact that it can only be specified with a single input and a single output.**

In [15]:
inputs = keras.Input(shape=(3,), name="my_input")
features = layers.Dense(64, activation="relu")(inputs)
outputs = layers.Dense(10, activation="softmax")(features)
model = keras.Model(inputs=inputs, outputs=outputs)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_input (InputLayer)       [(None, 3)]               0         
                                                                 
 dense_8 (Dense)             (None, 64)                256       
                                                                 
 dense_9 (Dense)             (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


**Multi-input, multi-output**

In [19]:
vocabulary_size = 10000
num_tags = 100
num_departments = 4

# Inputs
title = keras.Input(shape=(vocabulary_size,), name="title")
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")

# Hidden layer
features = layers.Concatenate()([title, text_body, tags]) # Concatenate inputs into single tensor
features = layers.Dense(64, activation="relu")(features)

# Outputs
priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(num_departments, activation="softmax", name="department")(features)

model = keras.Model(inputs=[title, text_body, tags], outputs=[priority, department])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 title (InputLayer)             [(None, 10000)]      0           []                               
                                                                                                  
 text_body (InputLayer)         [(None, 10000)]      0           []                               
                                                                                                  
 tags (InputLayer)              [(None, 100)]        0           []                               
                                                                                                  
 concatenate_2 (Concatenate)    (None, 20100)        0           ['title[0][0]',                  
                                                                  'text_body[0][0]',        